# Uczenie maszynowe z wykorzystaniem szyfrowania homomorficznego
### Autorzy: Marcin Szwed, Konrad Meling, Patryk Twardowski

## Przegląd algorytmów
* regresja liniowa, metody gradientowe - stosunkowo proste, wyłącznie mnożenie i dodawanie
* pełna liniowa regresja - jeżeli serwer zgromadziłby wszystkie dane od klientów konieczne jest odwracanie macierzy
* regresja z użyciem prostych sieci neuronowych np. z jedną warstwą ukrytą - teoretycznie możliwa, ale zawyczaj stosowane są nieliniowe funkcje aktywacji jak ReLU
* regresja logistyczna - odpada, bo exp()
* klasyfikacja naiwną metodą Bayesa - odpada, bo log(), chyba, że mały wymiar, bo wtedy można pomnożyć prawdopodobieństwa
* klasyfikacja metodą najbliższysch sąsiadów KNN - teoretycznie można obliczać w zaszyfrowany sposób odległość, ale musiałaby być porównywana po stronie klienta więc raczej bez sensu
* drzewa decyzyjne - wymagają porównań
* klasyfikacja z siecią neuronową - wymaga exp() w ostatniej warstwie, bo sigmoid albo softmax jest funkcją aktywacji ostatniej warstwy, funkcja celu zawiera zazwyczaj logarytm (cross entropy)

Właściwie zostaje regresja liniowa. W przypadku sieci neuronowych trzeba by stosować wielomianowe aproksymacje rozmaitych funkcji co jest teoretycznie możliwe.


## Organizacja uczenia
Przypadek A:
* jeden klient, jeden serwer
* klient szyfruje dane i model lokalnie i przesyła do serwera
* serwer wykonuje uczenie i zwraca klientowi metryki uczenia
* jeżeli klient chce dokonać predykcji to szyfruje dane, wysyła do serwera i dostaje rezultaty

Przypadek B:
* kilku niezależnych klientów ma swoje dane, którymi nie chce się dzielić
* jeden serwer przyjmuje zakodowane dane i używa ich do uczenia modelu
* model (wagi) jest zakodowany i przechowywany w takiej formie przez serwer
* na żądanie przez klienta serwer może wyznaczyć zaszyfrowaną predykcję dla zaszyfrowanych danych
* klient odkoduje predykcję

Problem: Trzeba ustalić kto ma jakie klucze, np. klucz publiczny ma serwer i klienci, klucze prywatne mają tylko klienci, klucz Galois i relinearyzacji ma serwer.

## Poniższy przykład
* klient ma parametry modelu i dane
* klient przesyła zaszyfrowane parametry modelu i dane do serwera
* serwer oblicza błąd i zwraca do klienta
* klient uaktualnia model

In [1]:
import tenseal as ts
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data = load_diabetes()
X = data.data[:, :3]  
y = data.target

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

X_train = X[:20]
y_train = y[:20]

context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2 ** 40
context.generate_galois_keys()


n_features = X_train.shape[1]
w = np.zeros(n_features)
b = 0.0
learning_rate = 0.1
epochs = 10

#Trening: klient-serwer
for epoch in range(epochs):
    grad_w = np.zeros(n_features)
    grad_b = 0.0

    # klient szyfruje parametry
    w_enc = ts.ckks_vector(context, w.tolist())
    b_enc = ts.ckks_vector(context, [b])

    
    for i in range(len(X_train)):
        x_i = X_train[i]
        y_i = y_train[i]
        # klient szyfruje dane
        x_enc = ts.ckks_vector(context, x_i.tolist())
        y_enc = ts.ckks_vector(context, [y_i])

        # Tu oblicza serwer
        pred_enc = x_enc.dot(w_enc) + b_enc

        error_enc = pred_enc - y_enc

        # serwer wysyła do klienta
        error = error_enc.decrypt()[0]

        grad_w += error * x_i
        grad_b += error

    grad_w /= len(X_train)
    grad_b /= len(X_train)

    w -= learning_rate * grad_w
    b -= learning_rate * grad_b

    y_pred = X_train.dot(w) + b
    mse = mean_squared_error(y_train, y_pred)
    print(f"Epoka {epoch+1}, MSE (HE): {mse:.5f}")


lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_train)
mse_lr = mean_squared_error(y_train, y_pred_lr)

print("MSE modelu HE: ", mean_squared_error(y_train, X_train.dot(w) + b))
print("MSE modelu sklearn: ", mse_lr)
print("Wagi HE:", w)
print("Wagi sklearn: ", lr.coef_)
print("Bias HE: ", b)
print("Bias sklearn: ", lr.intercept_)

Epoka 1, MSE (HE): 0.51257
Epoka 2, MSE (HE): 0.49286
Epoka 3, MSE (HE): 0.47592
Epoka 4, MSE (HE): 0.46118
Epoka 5, MSE (HE): 0.44824
Epoka 6, MSE (HE): 0.43677
Epoka 7, MSE (HE): 0.42657
Epoka 8, MSE (HE): 0.41743
Epoka 9, MSE (HE): 0.40922
Epoka 10, MSE (HE): 0.40182
MSE modelu HE:  0.4018181815793052
MSE modelu sklearn:  0.324313493798384
Wagi HE: [-0.14735328 -0.06858322  0.17461548]
Wagi sklearn:  [-0.32609907 -0.13320203  0.55650174]
Bias HE:  -0.09468727161194786
Bias sklearn:  -0.13157153183388606
